In [67]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

import json
import numpy as np
import cv2

In [77]:
cell_size=7
img_size = 112
classes = {'circle':0, 'triangle':1,  'rectangle':2}
VGG_MEAN = [123.68, 116.78, 103.94]

In [73]:
def load():
    labels = json.load(open('data/labels.json'))
    N = len(labels)
    X = np.zeros((N, img_size, img_size, 3), dtype='uint8')    
    y = np.zeros((N,cell_size, cell_size, 8))
    for idx, label in enumerate(labels):
        img = cv2.imread("data/{}.png".format(idx))
        X[idx] = img
        for box in label['boxes']:
            x1, y1 = box['x1'], box['y1']
            x2, y2 = box['x2'], box['y2']
            cl = [0]*len(classes)
            cl[classes[box['class']]] = 1
            
            x_center, y_center, w, h = (x1+x2)/2.0, (y1+y2)/2.0, x2-x1, y2-y1
            x_idx, y_idx = int(x_center/img_size*cell_size), int(y_center/img_size*cell_size)
            y[idx, y_idx, x_idx] = 1, x_center, y_center, w, h, *cl
    
    return X, y

In [74]:
X, y = load()

In [81]:
def vgg16(inputs):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.relu,
                      weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
        net = slim.max_pool2d(net, [2, 2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
        net = slim.max_pool2d(net, [2, 2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
        net = slim.max_pool2d(net, [2, 2], scope='pool5')
        net = slim.fully_connected(net, 4096, scope='fc6')
        net = slim.dropout(net, 0.5, scope='dropout6')
        net = slim.fully_connected(net, 4096, scope='fc7')
        net = slim.dropout(net, 0.5, scope='dropout7')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc8')
    return net

In [83]:
graph = tf.Graph()
with graph.as_default():    
    image = tf.placeholder("float", [None, img_size, img_size, 3], name="input")
    vgg = vgg16(image)
    variables_to_restore = tf.contrib.framework.get_variables_to_restore(exclude=['vgg_16/fc8'])
    init_fn = tf.contrib.framework.assign_from_checkpoint_fn('../vgg/vgg_16.ckpt', variables_to_restore)


In [84]:
variables_to_restore

[<tf.Variable 'conv1/conv1_1/weights:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv1/conv1_1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv1/conv1_2/weights:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv1/conv1_2/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2/conv2_1/weights:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv2/conv2_1/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2/conv2_2/weights:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'conv2/conv2_2/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv3/conv3_1/weights:0' shape=(3, 3, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv3/conv3_1/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3/conv3_2/weights:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'conv3/conv3_2/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3/conv3_3/weights:0' shape=(3, 3, 256, 256) dtype=float32_r